In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv')
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
drop_col=['PassengerId','Name','Ticket','Cabin','Embarked']
df=df.drop(drop_col,axis=1)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [6]:
df['Age']=df.fillna(df['Age'].mean())

In [7]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
dtype: int64

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,0,1,0,7.2500
1,1,1,female,1,1,0,71.2833
2,1,3,female,1,0,0,7.9250
3,1,1,female,1,1,0,53.1000
4,0,3,male,0,0,0,8.0500


In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Sex']=le.fit_transform(df['Sex'])

In [13]:
X=df.iloc[:,1:]
Y=df.iloc[:,0]
X.shape,Y.shape

((891, 6), (891,))

In [17]:
df.loc[1],X.loc[1]

(Survived          1
 Pclass            1
 Sex               0
 Age               1
 SibSp             1
 Parch             0
 Fare        71.2833
 Name: 1, dtype: object,
 Pclass          1
 Sex             0
 Age             1
 SibSp           1
 Parch           0
 Fare      71.2833
 Name: 1, dtype: object)

In [18]:
def entropy(col):
    N=float(col.shape[0])
    e=0.0
    count=np.unique(col,return_counts=True)
    for i in count[1]:
        p=i/N
        e+=(-1.0*p*np.log2(p))
    return e    

In [24]:
def divide(xd,fkey,fval):
    left=pd.DataFrame([],columns=xd.columns)
    right=pd.DataFrame([],columns=xd.columns)
    for ix in range(xd.shape[0]):
        val=xd[fkey].loc[ix]
        if val>fval:
            right=right.append(xd.loc[ix])
        else:
            left=left.append(xd.loc[ix])
    return left,right        

In [27]:
#l,r=divide(df[:10],'Sex',0.5)
#l,r

(  Survived Pclass Sex Age SibSp Parch     Fare
 1        1      1   0   1     1     0  71.2833
 2        1      3   0   1     0     0   7.9250
 3        1      1   0   1     1     0  53.1000
 8        1      3   0   1     0     2  11.1333
 9        1      2   0   1     1     0  30.0708,
   Survived Pclass Sex Age SibSp Parch     Fare
 0        0      3   1   0     1     0   7.2500
 4        0      3   1   0     0     0   8.0500
 5        0      3   1   0     0     0   8.4583
 6        0      1   1   0     0     0  51.8625
 7        0      3   1   0     3     1  21.0750)

In [28]:
def info(xd,fkey,fval):
    left,right=divide(xd,fkey,fval)
    if left.shape[0]==0 or right.shape[0]==0:
        return -100000
    l=left.shape[0]/xd.shape[0]
    r=right.shape[0]/xd.shape[0]
    ig=entropy(xd.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    return ig

In [29]:
for fx in X.columns:
    print(fx)
    print(info(df,fx,df[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.9607079018756469
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464
